In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,    # 숫자가 높을수록 랜덤함.
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()]
    )     

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat


In [5]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe":chef_chain} | veg_chain

final_chain.invoke({"cuisine":"indian"})

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare the tofu or paneer as a replacement for the chicken:\n\n**Tofu:**\n1. Use firm or extra firm tofu for this recipe. Press the tofu to remove excess water by wrapping it in a clean kitchen towel and placing a heavy object on top for about 15-20 minutes.\n2. Cut the tofu into bite-sized cubes and follow the marination process as you would with the chicken. Tofu absorbs flavors well, so marinate it for at least 1 hour, preferably longer.\n3. Instead of baking, you can pan-fry the marinated tofu in a bit of oil until it's golden brown and slightly crispy on the outside.\n\n**Paneer:**\n1. Paneer is a type of Indian cheese that holds its shape well when cooked. You can either buy it pre-made or make it at home by curdling hot milk with lemon juice or vinegar, then straining the curds.\n2. Cut the paneer into